# Similarity search

In [37]:
import re
import pandas as pd
import numpy as np
from tqdm import tqdm
from sklearn.svm import SVC
from sklearn import preprocessing, decomposition, model_selection, metrics, pipeline
from sklearn.model_selection import GridSearchCV
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.decomposition import TruncatedSVD
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score, classification_report
from sklearn import model_selection, preprocessing, metrics, linear_model, svm
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer

from imblearn.under_sampling import (RandomUnderSampler, 
                                    NearMiss, 
                                    InstanceHardnessThreshold,
                                    CondensedNearestNeighbour,
                                    EditedNearestNeighbours,
                                    RepeatedEditedNearestNeighbours,
                                    AllKNN,
                                    NeighbourhoodCleaningRule,
                                    OneSidedSelection,
                                    TomekLinks)
from imblearn.over_sampling import BorderlineSMOTE, SMOTE, ADASYN, SMOTENC, RandomOverSampler
from imblearn.combine import SMOTEENN, SMOTETomek
from imblearn.under_sampling import RandomUnderSampler
from imblearn.pipeline import make_pipeline as make_pipeline_imb
from imblearn.metrics import classification_report_imbalanced

import nltk
from nltk import word_tokenize
from nltk.corpus import stopwords
import qgrid

import sys
sys.path.insert(0, "..")

### After manually annotating text

In [4]:
train = pd.read_csv("result.csv")
train.dropna(0, subset=["text_clean", "sentiment"], inplace = True)
train = train[train["type"] != "title"].copy()
print(train.shape[0])

395


## Train, Valid and Test data

In [5]:
# Spliting into X & y
#X = train.iloc[:, 2].values # to get th
X = train["text_clean"].values # to get th

# Convert label to numeric
cleanup_label = {"sentiment": {"Yes": 1, "No": 0}}
train.replace(cleanup_label, inplace=True)
y = train.sentiment.values

In [59]:
# Split train and validation
from sklearn.model_selection import train_test_split
train_x, valid_x, train_y, valid_y = train_test_split(X, y, 
                                                  stratify=y, 
                                                  random_state=42, 
                                                  test_size=0.2, shuffle=True)

In [60]:
# label encode the target variable 
encoder = preprocessing.LabelEncoder()
train_y = encoder.fit_transform(train_y)
valid_y = encoder.fit_transform(valid_y)

## Models

In [40]:
# Create TF-IDF
tfidf_vect = TfidfVectorizer(min_df=3, max_features=100000, strip_accents='unicode',
                             analyzer='word', token_pattern=r'\w{1,}',ngram_range=(1, 3),
                             use_idf=1, smooth_idf=1, sublinear_tf=1)

# Fitting TF-IDF to both training and test sets (semi-supervised learning)
tfidf_vect.fit(X)
#tfv.fit(list(train_x) + list(valid_x))
xtrain_tfidf = tfv.transform(train_x)
xvalid_tfidf = tfv.transform(valid_x)

Check prediction with unbalanced classes

In [53]:
pipe = make_pipeline_imb(tfidf_vect,
                     MultinomialNB())
pipe.fit(train_x, train_y)
y_pred = pipe.predict(valid_x)

print(classification_report_imbalanced(valid_y, y_pred))

                   pre       rec       spe        f1       geo       iba       sup

          0       0.87      1.00      0.00      0.93      0.00      0.00       103
          1       0.00      0.00      1.00      0.00      0.00      0.00        16

avg / total       0.75      0.87      0.13      0.80      0.00      0.00       119



/Users/leonardovida/Library/Caches/pypoetry/virtualenvs/histaware-RplM6c3o-py3.8/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1221: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Try to balance classes

In [54]:
pipe = make_pipeline_imb(tfidf_vect,
                         RandomUnderSampler(),
                         MultinomialNB())
pipe.fit(train_x, train_y)
y_pred = pipe.predict(valid_x)

print(classification_report_imbalanced(valid_y, y_pred))

                   pre       rec       spe        f1       geo       iba       sup

          0       1.00      0.67      1.00      0.80      0.82      0.65       103
          1       0.32      1.00      0.67      0.48      0.82      0.69        16

avg / total       0.91      0.71      0.96      0.76      0.82      0.65       119



In [64]:
ros = RandomOverSampler(random_state=0)
xtrain_res, ytrain_res = ros.fit_resample(train_x, train_y)

ValueError: Expected 2D array, got 1D array instead:
array=['de gemeente mag heffen op als in de van het perceel riolering bestrating en en zijn aangebracht staatssecretaris van van stelt hiermee de criteria voor het maken van grond de ministerraad is hiermee gisteren akkoord gegaan'
 'onderzoekers zich konden vormen van het gebied onder de moet zich op zes kilometer diepte een bevinden van zich een recht naar boven naar het midden van de krater van die banen delen van magma zich een weg naar opzij en het gas dat vloeibare verzengend hete materiaal zoekt zich vervolgens langs breken een weg naar boven het zoeken van die wegen blijkt een maat te zijn voor al dan niet heftige onderaardse bewegingen die een doen zijn is dit alleen voor het geval of is het ook van toepassing op andere soortgelijke relaties en seismische activiteit hebben we al aangetoond voor op en en de verdenken we er ook van hoe tijdig het levensgevaarlijke gedrag van de genoemde zogeheten explosieve met deze gegevens k

In [55]:
pipe = make_pipeline_imb(tfidf_vect,
                         RandomOverSampler(),
                         MultinomialNB())
pipe.fit(train_x, train_y)
y_pred = pipe.predict(valid_x)

print(classification_report_imbalanced(valid_y, y_pred))

                   pre       rec       spe        f1       geo       iba       sup

          0       1.00      0.73      1.00      0.84      0.85      0.71       103
          1       0.36      1.00      0.73      0.53      0.85      0.75        16

avg / total       0.91      0.76      0.96      0.80      0.85      0.71       119



In [56]:
pipe = make_pipeline_imb(tfidf_vect,
                         SMOTE(),
                         MultinomialNB())
pipe.fit(train_x, train_y)
y_pred = pipe.predict(valid_x)

print(classification_report_imbalanced(valid_y, y_pred))

                   pre       rec       spe        f1       geo       iba       sup

          0       1.00      0.74      1.00      0.85      0.86      0.72       103
          1       0.37      1.00      0.74      0.54      0.86      0.76        16

avg / total       0.92      0.77      0.96      0.81      0.86      0.72       119



In [15]:
#Return the f1 Score
def train_model(classifier, feature_vector_train, label, feature_vector_valid):
    # fit the training dataset on the classifier
    classifier.fit(feature_vector_train, label)
    
    # predict the labels on validation dataset
    predictions = classifier.predict(feature_vector_valid)    

    return metrics.f1_score(valid_y,predictions)

In [21]:
#Return Predections Label
def train_model_Pred(classifier, feature_vector_train, label, feature_vector_valid, is_neural_net=False):
    # fit the training dataset on the classifier
    classifier.fit(feature_vector_train, label)
    
    # predict the labels on validation dataset
    predictions = classifier.predict(feature_vector_valid)
    
    if is_neural_net:
        predictions = predictions.argmax(axis=-1)
    
    return predictions

### Simple Multinomial NB

In [58]:
# Training the classifier & predicting on test data
classifier = MultinomialNB()
classifier.fit(xtrain_tfidf, train_y)
yvalid_pred = classifier.predict(xvalid_tfv)

AttributeError: lower not found

In [27]:
from sklearn.metrics import f1_score
f1_score(valid_y, yvalid_pred, average='weighted', labels=yvalid_pred)

0.9251700680272114

### Predict values and save

In [65]:
# Load the previous dataset for testing
test = pd.read_csv("../data/processed/selected_articles/1990s/to_label_gas.csv")
test.rename(columns={"label":"sentiment"}, inplace=True)
test["sentiment"] = None

# Select only new rows and not the training set
test_df = test.loc[~test.index.isin(train.index)]

# Drop na and titles
test_df.dropna(0, subset=["text_clean"], inplace = True)
test_df = test_df[test_df["type"] != "title"].copy()

In [79]:
x_test = test_df["text_clean"].values
y_test = pipe.predict_proba(x_test)

In [75]:
#Percentage of Positive/Negative
unique, counts = np.unique(y_test, return_counts=True)
dict(zip(unique, counts))

{0: 27335, 1: 14015}

In [80]:
test_df["sentiment"] = y_test

In [89]:
pd.set_option('display.max_colwidth', None) 
test_df[test_df["sentiment"] < 0.2]

,Unnamed: 0,type,sentiment,text,article_name,date,index_article,article_filepath,dir,metadata_title,...,newspaper_title,newspaper_date,newspaper_city,newspaper_publisher,newspaper_source,newspaper_volume,newspaper_issuenumber,newspaper_language,count,text_clean
141,141,p,0.019164,"*fr^ _______^ W\w^ _________ __E_______^^ Br_^B Ben je een eigenaardig bedrijf als je je klanten helpt om alleen het hoognodige van ••ggJJJgSBt y Jfl| __É^^^ jouw produkt te laten gebruiken? Op het eerste gezicht wel natuurlijk. jm Maar als we wat verder kijken en vooral verder denken, valt dat wel weer mee. Omdat .-mU 'AmW er vandaag de dag meer zaken tellen dan alleen economische. Milieu-aspecten bijvoorbeeld. ƒ l__r^ Voor een bedrijf als de Gasunie tellen die aspecten zeker mee. Want hoewel aardgas de /_lP____ V schoonste fossiele brandstof is, komen er bij de verbranding toch milieu-onvriendelijke stoffen vrij. ƒ -, Dat wetende besloot de Gasunie op eigen initiatief het Milieu Plan Industrie op te zetten. Een plan jj Jl I 1 m m \t _______________ _________________________ __m ________»mit ________ ■*** Bi _______t ■______. _____ ______ -**_____■ met de betreffende industrie lot optimale ■ m W W W I benutting van aardgas te komen. En *ï^k^k^_^_^J l^_____r I twee: om dc uitstoot van schadelijke H^;> stoffen tot een minimum te reduceren- _^^^^ Wat zowel economische voor_____ m _rm - — JH V VA l_^*^__ ___H__^""*""^__. __^V^ _^^^^k _L_^Hj^——^P_—l delen oplevert als positieve milieu-effecten heeft. H I lil II __P""ri_l H hi 3 jaar tijd gaat de Gasunie samen met de ■■MH _T^fe__(^ W__É """"'""'^ g1""001^ bedrijven van ons land hun stook- en industriële processen doorlichten. ■ I ■ Daarna adviseren we deze IraT^-i I _r^__ -..j^w. _^^^m 11^^ -W l_r^__ -^ ll^/__^^tl I __T^» de I _| I _| I energieverbruik te beperken en hun JB schadelijke emissies te verlagen. _^_^_^_. Om dit plan te realiseren, M_-__ neefl de Gasunie een bedrag van ruim 10 miljoen gulden uitgetrokken. Dat lijkt heel wat, maar als u weet dat elk Hk bedrijf anders is en de Gasunie maatwerk nastreeft, dan valt dat wel weer mee. __ Wv' -■■ kW wr^ 2 Tot nu toe lijken onze plannen verwezenlijkt te worden. Temeer omdat de industrie van harte meewerkt B* Neem Delamine, een joint venture van Akzo en het Japanse bedrijf Tosoh, die ethyleenaminen maakt voor talrijke industriële toepassingen en consumentenprodukten. jp*3*-""""' Bij Delamine blijkt door procesintegratie een energiebesparing mogelijk van 20%. Als u meer wilt weten _____P^^^ ■','■*.' wJk over het Milieu Plan Industrie, kunt u contact opnemen met ir. P.F.Jansen bij Delamine in Delfzijl (05960-19303) of _JJ_SjLJ|'f;Jfmm ■.- '■ ■—■ : k,rr' _____t C^nF^É^.J*_^|gM§ met ir. J.W.M, van Rijnsoever bij de Gasunie (050-213064). ■HV Jl Wij steken veel energie in energiebesparing. Gasunie . — . ■—-—i — i ' —",DDD_010646772_0118_articletext.xml,1992-03-26,2276808,/Users/leonardovida/dev/HistAware/data/raw/delpher/1990s/1992/03-26/DDD_010646772/DDD_010646772_0118_articletext.xml,/Users/leonardovida/dev/HistAware/data/raw/delpher/1990s/1992/03-26/DDD_010646772,DDD:ddd:010646772:mpeg21.didl.xml.gz.xml,...,De Telegraaf,1992-03-26,Amsterdam,Dagblad De Telegraaf,KB C 98,100.0,32399.0,nl,9,ben je een eigenaardig bedrijf als je je klanten helpt om alleen het hoognodige van te laten op het eerste gezicht wel maar als we wat verder kijken en vooral verder denken valt dat wel weer mee omdat er vandaag de dag meer zaken tellen alleen economische bijvoorbeeld voor een bedrijf als de tellen die aspecten zeker mee want hoewel aardgas de fossiele brandstof is komen er bij de verbranding toch stoffen vrij dat wetende besloot de op eigen initiatief het plan op te zetten een plan met de betreffende lot optimale van aardgas te komen en twee om van schadelijke stoffen tot een te wat zowel economische va delen oplevert als positieve heeft lil jaar tijd de samen met de bedrijven van ons land en processen doorlichten daarna adviseren we deze de te beperken en schadelijke te 

In [75]:
# Predict on new values
test = test_df["text_clean"].values
xtest_tfv = tfv.transform(test)
y_test = clf.predict_proba(xtest_tfv)

In [76]:
# Add prediction to df
test_df["sentiment"] = y_test

42406

In [94]:
# Only select those with 1 as label
gas_labeled = test_df[test_df["sentiment"] < 0.8]
gas_labeled.shape[0]

451

In [36]:
# Display, seems to work decently
pd.set_option('display.max_colwidth', None) 
test_df[["text", "sentiment"]].head(100)

text  \
270                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                          Het is vrijdag 9 december 31 jaar geleden dat uit Slochteren het eerste aardgas door de Nederlandse Aardolie Maatschappij (NAM) aan de i Gasunie werd geleverd. In 1949 werd het grote Groninger gasveld ontdekd Sindsdien speelt aardgas nergens in de wereld zon belangrijke rol als in Nederland. De helft van de totale energiebehoefte in ons land 'drijft' er op. De NCRV staat er morgenochtend (van 9.00 tot 11.00 uur op radio 5) tijdens Het Gesprek en Plein Publiek uitgebreid bij stil. Ton Alberts, de architect van het nieuwe Gasunie-gebouw in Groningen, is te gast in Het Gesprek. Vanuit de markante kolos van de Gasunie wordt in Plein Publiek gesproken met een aantal gasten — waaronder Commissaris der Koningin H.J.L. Vonhoff - 

### TFIDF - Class

In [34]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.svm import SVC
from sklearn.naive_bayes import GaussianNB
from sklearn.naive_bayes import MultinomialNB
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.metrics import precision_score, recall_score, f1_score, accuracy_score
import warnings
import pandas as pd


def Vectorize(vec, X_train, X_test):

    X_train_vec = vec.fit_transform(X_train)
    X_test_vec = vec.transform(X_test)

    print("Vectorization complete.\n")

    return X_train_vec, X_test_vec


# Use multiple classifiers and grid search for prediction
def ML_modeling(models, params, X_train, X_test, y_train, y_test):

    if not set(models.keys()).issubset(set(params.keys())):
        raise ValueError("Some estimators are missing parameters")

    for key in models.keys():

        model = models[key]
        param = params[key]
        gs = GridSearchCV(model, param, cv=10, error_score=0, refit=True)
        gs.fit(X_train, y_train)
        y_pred = gs.predict(X_test)

        # Print scores for the classifier
        print(key, ":", gs.best_params_)
        print(
            "Accuracy: %1.3f \tPrecision: %1.3f \tRecall: %1.3f \t\tF1: %1.3f\n"
            % (
                accuracy_score(y_test, y_pred),
                precision_score(y_test, y_pred, average="macro"),
                recall_score(y_test, y_pred, average="macro"),
                f1_score(y_test, y_pred, average="macro"),
            )
        )

    return


## Preparing to make a pipeline
models = {
    "Naive Bayes": MultinomialNB(),
    "Gradient Boosting": GradientBoostingClassifier(),
}

params = {
    "Naive Bayes": {"alpha": [0.5, 1], "fit_prior": [True, False]},
    "Gradient Boosting": {"learning_rate": [0.05, 0.01], "min_samples_split": [2, 5]},
}

In [35]:
# Train-test split and vectorize
X_train, X_test, y_train, y_test = train_test_split(
    train["text_wop"], train["sentiment"], test_size=0.3, shuffle=True
)

X_train_vec, X_test_vec = Vectorize(TfidfVectorizer(min_df=3, max_features=None,
                      strip_accents='unicode', analyzer='word',token_pattern=r'\w{1,}',
                      ngram_range=(1, 2), use_idf=1,smooth_idf=1,sublinear_tf=1), X_train, X_test)

ML_modeling(models, params, X_train_vec, X_test_vec, y_train, y_test)

Vectorization complete.

Naive Bayes : {'alpha': 1, 'fit_prior': True}
Accuracy: 0.905 	Precision: 0.889 	Recall: 0.929 		F1: 0.899

Gradient Boosting : {'learning_rate': 0.01, 'min_samples_split': 2}
Accuracy: 0.762 	Precision: 0.757 	Recall: 0.679 		F1: 0.692

